In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install readlif

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from readlif.reader import LifFile
import glob
import cv2
import natsort

### Code to convert .Lif file to FPS

In [ ]:
def grayscale_to_color(image):
  width, height = image.shape
  color_image = np.zeros((width, height, 3), dtype=np.uint8)
  for i in range(width):
    for j in range(height):
      gray_value = image[i, j]
      color_image[i, j, 0] = 0
      color_image[i, j, 1] = gray_value * 2.2
      color_image[i, j, 2] = 0

  image = Image.fromarray(color_image)
  return image

In [ ]:
dataset_file = LifFile('/content/drive/MyDrive/Dissertation/Data/230221modERKlive.lif')
savehere_dir = '/content/drive/MyDrive/Dissertation/Data/FPSData/{}'

In [ ]:
z_values_videos = [10, 6, 6, 5, 9, 6, 4, 4, 4, 5]
video_index = 1
for video_file in dataset_file.get_iter_image():
  image_index = 1
  for frame in video_file.get_iter_t(z = z_values_videos[video_index - 1], c = 0):
    color_image = grayscale_to_color(np.array(frame))
    path = savehere_dir.format(video_index)
    if not os.path.exists(path):
      os.makedirs(path)
    color_image.save(path + '/{}.jpeg'.format(image_index))
    image_index += 1
  video_index += 1

### Code to convert FPS to Video

In [ ]:
fps_dir = '/content/drive/MyDrive/Dissertation/Data/FPSData/'
desti_dir = '/content/drive/MyDrive/Dissertation/Data/VideoDataset/'

In [ ]:
sub_folder = glob.glob(fps_dir + '*', recursive = True)

In [ ]:
sub_folder

['/content/drive/MyDrive/Dissertation/Data/FPSData/1',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/2',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/3',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/4',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/5',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/6',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/7',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/8',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/9',
 '/content/drive/MyDrive/Dissertation/Data/FPSData/10']

In [ ]:
for dir in sub_folder:
  file_name = dir.split('/')[-1]
  dir_temp = dir + '/*.jpeg'
  img_array = []
  for filename in natsort.natsorted(glob.glob(dir_temp)):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
  out = cv2.VideoWriter(desti_dir + '{}.avi'.format(file_name),
                        cv2.VideoWriter_fourcc(*'DIVX'),
                        3,
                        size)
  for i in range(len(img_array)):
      out.write(img_array[i])
  out.release()